In [116]:
# === Header: Setup SQLite and Connect to DB ===
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect('pokemon_data.db')  # or whatever your DB file is named
cursor = conn.cursor()

# Helper function to run SQL queries and show results as DataFrame
def run_query(query):
    return pd.read_sql_query(query, conn)

# Example usage (you can delete this later)
run_query("""
    SELECT * FROM pokemon""")


,id,name,generation,type1,type2,hp,attack,defense,special_attack,special_defense,speed,bst,stage,is_fully_evolved,dex_entry
0,1,bulbasaur,1,grass,poison,45,49,49,65,65,45,318,1.0,0.0,A strange seed was planted on its back at birt...
1,2,ivysaur,1,grass,poison,60,62,63,80,80,60,405,2.0,0.0,"When the bulb on its back grows large, it appe..."
2,3,venusaur,1,grass,poison,80,82,83,100,100,80,525,3.0,1.0,The plant blooms when it is absorbing solar en...
3,4,charmander,1,fire,None,39,52,43,60,50,65,309,1.0,0.0,"Obviously prefers hot places. When it rains, s..."
4,5,charmeleon,1,fire,None,58,64,58,80,65,80,405,2.0,0.0,"When it swings its burning tail, it elevates t..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1021,raging-bolt,9,electric,dragon,125,73,91,137,89,75,590,1.0,1.0,It's said to incinerate everything around it w...
1021,1022,iron-boulder,9,rock,psychic,90,120,80,68,108,124,590,1.0,1.0,It resembles a Pokémon described in a dubious ...
1022,1023,iron-crown,9,steel,psychic,90,72,100,122,108,98,590,1.0,1.0,It resembles a mysterious object introduced in...
1023,1024,terapagos,9,normal,None,90,65,85,65,85,60,450,1.0,1.0,Terapagos protects itself using its power to t...


# Analysis of Pokémon Data

This project investigates the factors contributing to a Pokémon's perceived "strength" by examining its statistics, types, and generational origin. Pokémon is a widely popular game featuring creatures known as Pokémon, which engage in combat utilizing various elemental types such as Fire, Water, and Electric. Each Pokémon is defined by six primary statistics:

- **HP** (Hit Points) – durability against damage
- **Attack** – offensive capability with physical attacks
- **Defense** – defensive capability against physical attacks
- **Sp. Atk** – offensive capability with special attacks
- **Sp. Def** – defensive capability against special attacks
- **Speed** – determines action order in combat
    The sum of these six statistics constitutes a Pokémon’s **Base Stat Total (BST)**, which is commonly employed as a metric for assessing overall capability.
    There are currently **9 generations** of Pokémon, each introducing new creatures, types, and mechanics. This analysis aims to address questions such as:
- Which Pokémon exhibit the highest statistical potential?
- Which type exhibits the most effective offensive capability (inflicts the highest amount of super-effective damage)?
- How do different types and generations compare in terms of potential strength?
    Visualizations and statistical methods are utilized to uncover patterns and insights across a comprehensive dataset of Pokémon.

## Identifying the Strongest Pokémon

Determining the "strongest" Pokémon can be approached using various criteria.

1. Pokémon with the highest Base Stat Total (BST) could be considered strongest, in which case the following Pokémon are candidates:

In [117]:
run_query("""
    SELECT id, name, bst
    FROM pokemon
    ORDER by bst desc
    LIMIT 5
    """)

,id,name,bst
0,493,arceus,720
1,890,eternatus,690
2,150,mewtwo,680
3,249,lugia,680
4,250,ho-oh,680


Utilizing BST as the criterion, Arceus emerges as the leading contender. However, all five of these Pokémon are classified as Legendary, typically encountered only within the narrative context of the game's storyline due to their significance to the lore. Their exceptional strength is often attributed to this narrative positioning.
Regrettably, the available data lacks a specific attribute for filtering based on Legendary status. However, filtering by evolution stage is feasible. Numerous Pokémon undergo evolution throughout the game's progression; conversely, most Legendary Pokémon do not evolve. Consequently, examining Pokémon with the highest BST that possess more than one evolution stage may yield more relevant insights:

In [118]:
run_query("""
    SELECT id, name, bst
    FROM pokemon
    WHERE stage != 1
    ORDER by bst desc
    LIMIT 10
    """)

,id,name,bst
0,791,solgaleo,680
1,792,lunala,680
2,289,slaking,670
3,149,dragonite,600
4,248,tyranitar,600
5,373,salamence,600
6,376,metagross,600
7,445,garchomp,600
8,490,manaphy,600
9,635,hydreigon,600


As observed, this table largely excludes Legendary Pokémon. Only Solgaleo, Lunala, and Manaphy remain, as they are exceptional cases possessing evolution stages. This represents the closest feasible method within the data to approximate the exclusion of Legendary Pokémon.

2. Alternatively, a Pokémon's 'strength' could be assessed based on its offensive power. Effective offense often depends on either initiating action first (due to high Speed) or enduring the opponent's attack. For analytical simplicity, this examination will prioritize the scenario where the Pokémon acts first, as this may mitigate or negate incoming damage. The average Speed statistic for fully evolved Pokémon is calculated as follows:

In [119]:
run_query("""
    SELECT avg(speed) as avg_speed
    FROM pokemon
    WHERE is_fully_evolved = 1
    """)

,avg_speed
0,78.136194


Subsequently, Pokémon possessing the highest Attack or Special Attack statistics and a Speed value equal to or exceeding this average are identified:

In [120]:
run_query("""
    SELECT id, name, attack, speed
    FROM pokemon
    WHERE is_fully_evolved = 1 AND speed > (select avg(speed) from pokemon)
    ORDER by attack desc
    LIMIT 10
    """)

,id,name,attack,speed
0,798,kartana,181,109
1,289,slaking,160,100
2,486,regigigas,160,100
3,383,groudon,150,90
4,384,rayquaza,150,95
5,644,zekrom,150,90
6,612,haxorus,147,97
7,998,baxcalibur,145,87
8,567,archeops,140,110
9,794,buzzwole,139,79


In [121]:
run_query("""
    SELECT id, name, special_attack, speed
    FROM pokemon
    WHERE is_fully_evolved = 1 AND speed > (select avg(speed) from pokemon)
    ORDER by special_attack desc
    LIMIT 10
    """)

,id,name,special_attack,speed
0,796,xurkitree,173,83
1,150,mewtwo,154,130
2,806,blacephalon,151,107
3,382,kyogre,150,90
4,384,rayquaza,150,95
5,483,dialga,150,90
6,484,palkia,150,100
7,643,reshiram,150,90
8,720,hoopa,150,70
9,609,chandelure,145,80


These Pokémon could also be regarded as exceptionally capable, given their superior speed relative to the average and their potent offensive statistics.

## Analyzing Pokémon by Type
Pokémon are assigned elemental types, frequently possessing two types concurrently, referred to as a type combination. Type assignments dictate resistances and vulnerabilities; for instance, Fire-type Pokémon are susceptible to increased damage from Water-type attacks, whereas Grass-type Pokémon exhibit resistance to them. Type effectiveness modifiers also interact. Consequently, a Pokémon possessing both Grass and Fire typing would receive standard damage from Water-type attacks. A Type weakness results in a damage multiplier of two times the standard amount, while resistance reduces damage to half. Mutual weakness across both types yields a quadruple damage multiplier, whereas mutual resistance results in one-fourth the standard damage.

Certain Pokémon types are considered to possess superior offensive capabilities compared to others. The Ice type stands out as a prime example, possessing significant offensive strength against Ground, Flying, and Dragon types. A key characteristic of these three types is their limited number of weaknesses; each possesses only three. Notably, the Ice type represents a unique intersection, being the sole shared weakness among Ground, Flying, and Dragon. However, beyond these specific type interactions, the data presented below clearly illustrates the substantial number of Pokémon across the board that are susceptible to Ice-type attacks:

In [122]:
run_query("""
    WITH AllAttackingTypes AS (
    SELECT
        id AS attacking_type_id,
        name AS attacking_type_name -- Assuming type.name is consistently cased (e.g., "Grass", "Fire")
    FROM type
),
PokemonEffectivenessAcrossAllAttacks AS (
    SELECT
        p.id AS pokemon_id,
        att.attacking_type_id,
        att.attacking_type_name,
        CASE
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t1.no_damage_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t1.weak_to) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 2.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t1.resist_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.5
            ELSE 1.0
        END AS type1_multiplier,
        CASE
            WHEN p.type2 IS NULL THEN 1.0
            WHEN t2.name IS NULL THEN 1.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t2.no_damage_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t2.weak_to) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 2.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t2.resist_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.5
            ELSE 1.0
        END AS type2_multiplier
    FROM
        pokemon p
    -- MODIFIED JOIN CONDITIONS:
    JOIN type t1 ON LOWER(p.type1) = LOWER(t1.name)       -- Case-insensitive join for Type 1
    LEFT JOIN type t2 ON LOWER(p.type2) = LOWER(t2.name)  -- Case-insensitive join for Type 2
    CROSS JOIN AllAttackingTypes att
),
PokemonTotalEffectivenessAcrossAllAttacks AS (
    SELECT
        pokemon_id,
        attacking_type_id,
        attacking_type_name,
        type1_multiplier * type2_multiplier AS total_multiplier
    FROM PokemonEffectivenessAcrossAllAttacks
)
SELECT
    att_summary.attacking_type_name AS "Attacking Type",
    SUM(CASE WHEN att_summary.total_multiplier = 2.0  THEN 1 ELSE 0 END) AS "Pokemon 2x Weak To",
    SUM(CASE WHEN att_summary.total_multiplier = 4.0  THEN 1 ELSE 0 END) AS "Pokemon 4x Weak To",
    SUM(CASE WHEN att_summary.total_multiplier = 0.5  THEN 1 ELSE 0 END) AS "Pokemon Resist (0.5x)",
    SUM(CASE WHEN att_summary.total_multiplier = 0.25 THEN 1 ELSE 0 END) AS "Pokemon 0.25x Resist",
    SUM(CASE WHEN att_summary.total_multiplier = 0.0  THEN 1 ELSE 0 END) AS "Pokemon Immune To"
FROM PokemonTotalEffectivenessAcrossAllAttacks att_summary
GROUP BY
    att_summary.attacking_type_id, att_summary.attacking_type_name
ORDER BY
    "Pokemon 2x Weak To" desc;
        """)

,Attacking Type,Pokemon 2x Weak To,Pokemon 4x Weak To,Pokemon Resist (0.5x),Pokemon 0.25x Resist,Pokemon Immune To
0,Ice,263,29,266,11,0
1,Ground,248,26,137,6,109
2,Rock,245,27,170,6,0
3,Fire,244,18,277,25,0
4,Fighting,241,17,230,52,65
5,Flying,240,13,158,12,0
6,Electric,208,8,210,9,75
7,Bug,170,14,314,63,0
8,Grass,168,29,346,102,0
9,Water,164,14,287,13,0


Based on the table presented above, it is evident that Ice-type moves deal double damage (2x) against 263 Pokémon. This figure represents the highest count in this category, placing Ice-type moves in the top position for 2x effectiveness. Furthermore, these moves inflict quadruple damage (4x) against 29 Pokémon, a count that ties with Grass-type moves for the highest number of Pokémon susceptible to 4x damage. Conversely, Ice-type moves are only half effective (0.5x damage) against 266 Pokémon, ranking sixth in terms of the number of Pokémon resistant to the type.

## Determining the Generally Best Pokémon Type
When considering the determination of the generally best Pokémon type, sole offensive strength, such as that exhibited by the Ice type, may not be sufficient criteria. Therefore, it is necessary to examine additional factors, including the average Base Stat Total (BST) of Pokémon belonging to each type.

In [123]:
run_query("""
    SELECT
        t.name AS type_name,
        ROUND(AVG(p.bst),2) AS avg_bst
    FROM
        pokemon p
    JOIN
        type t
    ON
        LOWER(p.type1) = LOWER(t.name) OR LOWER(p.type2) = LOWER(t.name)
    GROUP BY
        t.name
    ORDER BY
        avg_bst DESC
""")

,type_name,avg_bst
0,Dragon,504.76
1,Steel,479.72
2,Fighting,467.14
3,Dark,459.59
4,Psychic,457.45
5,Ice,453.65
6,Fire,449.51
7,Ghost,447.46
8,Electric,442.23
9,Rock,442.12


Based on the data presented above, the Dragon type appears to be the strongest based on average BST, boasting a mean BST of 504. This figure surpasses that of Steel, which ranks second, by a margin of 20 points. However, Dragon-type attacks are super effective against only 67 Pokémon, placing them in the seventeenth position in terms of super-effective targets, alongside the Normal type, which cannot deal super-effective damage to any type. Consequently, the strength of the Dragon type appears to be primarily attributed to its statistical profile rather than its offensive type effectiveness.

For those wondering why moves typing affect the process of ranking, the Same Type Attack Bonus (STAB) is a mechanic that grants a 1.5x damage multiplier when a Pokémon uses an attack that matches its own type. This bonus represents a significant factor in overall damage calculation.

The Steel type, occupying the second position in terms of average BST, while Steel-type attacks are super effective against approximately 140 Pokémon, is widely regarded as one of the premier defensive types. It possesses a notable 10 resistances (Normal, Grass, Ice, Flying, Psychic, Bug, Rock, Dragon, Fairy, and Steel), while being susceptible to only three types: Fire, Fighting, and Ground. Given its strong second-place ranking in average BST and its status as one of the most resilient defensive types, it can be argued that the Steel type is the generally best type overall, primarily due to its exceptional defensive profile and robust statistical average. The Dragon type follows closely in second place, distinguished by its impressive statistical base.

## Identifying the Generally Weakest Pokémon Type
In this instance, the average Base Stat Total (BST) ranking aligns with other indicators to identify the generally weakest type. The Bug type exhibits the lowest average BST at 374, standing as the sole type below the 400 threshold. Furthermore, Bug-type attacks demonstrate limited offensive efficacy: they deal reduced damage (0.5x) to a significant 7 types (Fire, Fighting, Poison, Flying, Ghost, Steel, and Fairy), while being super effective (2x) against only 3 types. The combination of its low average statistical profile and poor offensive type coverage strongly positions the Bug type as the weakest overall.

## Determining the Strongest/Weakest Pokémon Generation
Unlike some previous findings, the identification of the generation containing the strongest or weakest Pokémon can be readily determined by utilizing the average Base Stat Total (BST) metric. For the purpose of this analysis, Pokémon are assigned to the generation in which they first appeared. This methodology counts Pokémon debuting in a particular generation under that generation, even if they reappear in subsequent titles. While move pools may evolve and spread across generations, focusing on the average BST provides a consistent measure of the statistical baseline introduced by each generation.

In [ ]:
run_query("""
    SELECT
        generation AS Generation,
        ROUND(AVG(bst), 2) AS Average_BST
    FROM pokemon
    WHERE
        is_fully_evolved = 1
        AND stage != 1
    GROUP BY generation
    ORDER BY generation
""")

,Generation,Average_BST
0,1,493.64
1,2,486.08
2,3,484.18
3,4,518.86
4,5,507.20
5,6,518.65
6,7,525.02
7,8,516.75
8,9,534.39
